In [2]:
from Rings import Zomega, Exact_U, Ztau
from Circuits import FTCircuit
import numpy as np
import cmath

In [3]:
omega = Zomega(0,1,0,0)
tau = Zomega(0,0,1,-1)

T = Exact_U(Zomega(1,0,0,0), Zomega(0,0,0,0), 6)
F = Exact_U(tau, Zomega(1,0,0,0), 0)


In [8]:
def exact_synthesize(U):
    g = U.mu()
    V = U
    Circuit = FTCircuit()
    while g.value() >= 2: # need to be careful this doesn't break from rounding error? if it does change to Gauss complexity which is always an integer
        
        min = 0
        # print("V = " + str(V))
        for j in range(0,10):
            #print("j = " + str(j) + ", F*T**j*V = " + str(F*(T**j)*V))
            #print("mu = " + str((F*(T**j)*V).mu().value()))
            if (F*(T**j)*V).mu().value() < (F*(T**min)*V).mu().value():
                min = j

        V = F*(T**min)*V
        g = V.mu()
        if min != 0:
            Circuit.join("T", 10 - min)
        Circuit.join("F", 1)
    
    # first factor out the phase of omega^k
    for k in range(10):
        if omega**k == V.u:
            Circuit.addPhase(omega**k)
            V = Exact_U(omega**(10-k), Zomega(0,0,0,0), 5 - 2*k) * V
            break
    
    # Am left with a matrix of the form U(1, 0, k) which we can match to T^j
    if V.k%10 != 5:
        Circuit.join("T",(V.k)%10)
    
    return Circuit

(-8903304 + -1277765128ω + 2081873256ω² + -1292170978ω³), (997605933 + -1662824027ω + 1076345486ω² + -48663720ω³)

In [5]:
u, v = Zomega(-8903304, -1277765128, 2081873256, -1292170978), Zomega(997605933, -1662824027, 1076345486, -48663720)
                                    

(1 + 0ω + 0ω² + 0ω³)


In [10]:
uj = tau * omega**3
vi = omega**2
U = Exact_U(u, v, 0)

print(exact_synthesize(U).numerical_approximate())
#print(exact_synthesize(U).toBraid())
#print(exact_synthesize(U).toBraid().numerical_approximate())

[[0.707121814994159 - 0.706977325068627*I
  0.0123585793537003 + 0.00301099892733085*I]
 [-0.0123585793536996 + 0.00301099892733204*I
  0.707121814994163 + 0.706977325068624*I]]


In [21]:
def exact_synthesize_weave(U):
    g = U.mu()
    V = U
    Circuit = FTCircuit()
    Remainder = FTCircuit()
    f_parity = True

    while g.value() >= 4: # need to be careful this doesn't break from rounding error? if it does change to Gauss complexity which is always an integer
        
        min = 0
        print("V = " + str(V))
        for j in range(0,5):
            if (F*(T**(2*j))*V).mu().value() < (F*(T**min)*V).mu().value():
                min = 2*j

        V = F*(T**min)*V
        g = V.mu()
        if min != 0:
            Circuit.join("T", 10 - min)
        Circuit.join("F", 1)
        f_parity = not f_parity
    
    # check if |u| = 1
    if V.u*V.u.star() == Zomega(1,0,0,0):
        # factor out phase so that u = 1
        for k in range(10):
            if omega**k == V.u:
                Remainder.addPhase(omega**k)
                V = Exact_U(omega**(10-k), Zomega(0,0,0,0), 5 - 2*k) * V
                break
        
        # Am left with a matrix of the form U(1, 0, k) which we can match to T^j, for j = k-5
        # if j is even and f_parity = True, V is a weave
        if V.k%10 != 5:
            if f_parity:
                if (V.k-5)%2 == 0: 
                    Circuit.join("T", V.k-5)
                    Circuit.addPhase(omega**k)
            else:
                Remainder.join("T", V.k-5)
            
                
    # otherwise |u| = tau
    # We write V = U[w^j t, w^i, k] and find i,j,k
    elif Ztau.fromOmega(V.u*V.u.star()) == Ztau(1, -1):

        # find v
        for i in range(10):
            if omega**i == V.v:
                break

        # find j
        for j in range(10):
            if (omega**(10-j))*V.u == Zomega(0,0,1,-1):
                break
        
        # find k
        k = V.k

        print((j,i,k))

        # now V = w^j T^(-i-j) F T^(-i-j+k)
        if not f_parity:
            if (i-j)%2 == 0 and (-i-j+k)%2 == 0:
                Circuit.join("T",i-j)
                Circuit.join("F", 1)
                Circuit.join("T",-i-j+k)
                Circuit.addPhase(omega**j)

                return (Circuit, Remainder)
        
        if not f_parity:
            if Circuit.remove_F():
                Remainder.join("F", 1)
                
        Remainder.join("T",i-j)
        Remainder.join("F", 1)
        Remainder.join("T",-i-j+k)
        Remainder.addPhase(omega**j)
            
        
    else:
        print("meep! error: V.u not as expected")
    
    return (Circuit, Remainder)

In [22]:
U = T**2 * F * T**4 * F
print(U)

(circuit, remainder) = exact_synthesize_weave(U)
print(circuit)
print(remainder)
#print(exact_synthesize(U).toBraid())
#print(exact_synthesize(U).toBraid().numerical_approximate())

U((1 + -1ω + 0ω² + 1ω³),(1 + 0ω + 0ω² + 1ω³),1)
V = U((1 + -1ω + 0ω² + 1ω³),(1 + 0ω + 0ω² + 1ω³),1)
(0, 4, 4)
(1 + 0ω + 0ω² + 0ω³)(T^0)(F)(T^4)(F)(T^2)
(1 + 0ω + 0ω² + 0ω³)


In [19]:
exact_omega = Exact_U(omega, Zomega(0,0,0,0), 2)
V = exact_omega**2 * T**9 * F * T**7
print(V)

U((0 + 1ω + -1ω² + 1ω³),(0 + 1ω + 0ω² + 0ω³),0)
